In [5]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np
import pandas as pd
import os
import matplotlib
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook
%matplotlib inline 
import cv2 as cv
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.models import Sequential 
from keras.layers import Conv2D,MaxPool2D,DepthwiseConv2D,Flatten,Dense,Dropout,AveragePooling2D,BatchNormalization,GlobalAveragePooling2D
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.utils import to_categorical
from tensorflow.keras import regularizers

In [2]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
img_height=512
img_width=512
batch_size=64
directory="../input//dl-hack-track-1-cv/train/"

In [9]:
datagen=ImageDataGenerator(
    rescale=1.0/255,
#     featurewise_center=True,
#     featurewise_std_normalization=True,
#     rotation_range=20,
#     width_shift_range=0.2,
#     height_shift_range=0.2,
    horizontal_flip=False,
    data_format="channels_last",
    validation_split=0.2,
    dtype=tf.float32
    
    
)

In [10]:
train_data=datagen.flow_from_directory(
    "../input//dl-hack-track-1-cv/train/",
    target_size=(512,512),
    batch_size=batch_size,
    color_mode="rgb",
    class_mode='categorical',
    shuffle=True,
    subset='training'
    
)
val_data=datagen.flow_from_directory(
    "../input//dl-hack-track-1-cv/train/",
    target_size=(512,512),
    batch_size=batch_size,
    color_mode="rgb",
    class_mode='categorical',
    shuffle=True,
    subset='validation'
    
)


Found 28000 images belonging to 2 classes.
Found 7000 images belonging to 2 classes.


In [19]:
model= keras.Sequential()
# input shape is shape of images
model.add(keras.Input(shape=(512,512,3)))
model.add(Conv2D(32,(2,2),padding='same'))
#now step down the image dimentions 
model.add(MaxPool2D(pool_size=(2,2)))
model.add(BatchNormalization())
# from here the MTCNN architechure is take with slight modeification of channnels
model.add(Conv2D(32,(3,3),activation = 'relu',padding='same')) 
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPool2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(MaxPool2D((2, 2)))
model.add(Conv2D(256, (2, 2), activation='relu', padding='same'))
model.add(Conv2D(256, (2, 2), activation='relu', padding='same'))
model.add(MaxPool2D((2, 2)))
model.add(BatchNormalization())
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(10,activation='relu'))
model.add(Dense(2, activation='sigmoid'))
model.compile(
        optimizer=tf.keras.optimizers.Adam(0.000005),
            loss='categorical_crossentropy',
            metrics=['accuracy']
    )
model.summary()

model.fit(train_data,epochs=4,validation_data=val_data)

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_28 (Conv2D)           (None, 512, 512, 32)      416       
_________________________________________________________________
max_pooling2d_21 (MaxPooling (None, 256, 256, 32)      0         
_________________________________________________________________
batch_normalization_13 (Batc (None, 256, 256, 32)      128       
_________________________________________________________________
conv2d_29 (Conv2D)           (None, 256, 256, 32)      9248      
_________________________________________________________________
max_pooling2d_22 (MaxPooling (None, 128, 128, 32)      0         
_________________________________________________________________
conv2d_30 (Conv2D)           (None, 128, 128, 64)      18496     
_________________________________________________________________
batch_normalization_14 (Batc (None, 128, 128, 64)     

In [20]:
model.fit(train_data,epochs=4,validation_data=val_data)

Epoch 1/4
438/438 [==============================] - 1089s 2s/step - loss: 0.1256 - accuracy: 0.9513 - val_loss: 0.1033 - val_accuracy: 0.9587
Epoch 3/4
438/438 [==============================] - 1107s 3s/step - loss: 0.1088 - accuracy: 0.9589 - val_loss: 0.0953 - val_accuracy: 0.9630
Epoch 4/4
438/438 [==============================] - 1109s 3s/step - loss: 0.0966 - accuracy: 0.9648 - val_loss: 0.0916 - val_accuracy: 0.9659


In [ ]:
def inflow(i):
    x=MaxPool2D((2, 2))(i)
    x=DepthwiseConv2D((3,3),padding="same")(x)
    x=BatchNormalization()(x)
    #x=Conv2D(3,(3,3),padding="same",activation="relu")(x)
    return x
def outflow(x):
    x=Flatten()(x)
    x=Dense(64,activation='relu')(x)
    x=Dense(16,activation='relu')(x)
    x=Dense(2,activation="sigmoid")(x)
    return x
    
def midflow(x):
    x=Conv2D(filters=64,kernel_size=(3,3),padding="same", activation="relu")(x)
    x=Conv2D(filters=64,kernel_size=(3,3),padding="same", activation="relu")(x)
    x=MaxPool2D(pool_size=(2,2),strides=(2,2))(x)
    
    #========================================================================
    y=Conv2D(filters=64,kernel_size=(1,1), padding="same", activation="relu")(x)
    y=DepthwiseConv2D(kernel_size=(2,2),strides=(2,2), padding="same", activation="relu")(y)

    #=========================================================================
    x=Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu")(x)
    x=DepthwiseConv2D(kernel_size=(3,3), padding="same", activation="relu")(x)
    x=MaxPool2D(pool_size=(2,2),strides=(2,2))(x)
    x=layers.Concatenate()([x,y])

    x=Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu")(x)
    x=Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu")(x)
    x=Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu")(x)
    x=MaxPool2D(pool_size=(2,2),strides=(2,2))(x)
    
    #============================================================================\
    
    y=Conv2D(filters=64, kernel_size=(3,3), padding="same", activation="relu")(y)
    y=Conv2D(filters=64, kernel_size=(3,3), padding="same", activation="relu")(y)
    y=DepthwiseConv2D(kernel_size=(2,2),strides=(2,2), padding="same", activation="relu")(y)
    y=MaxPool2D(pool_size=(2,2),strides=(2,2))(y)
    
    
    z=Conv2D(filters=256,kernel_size=(1,1), padding="same", activation="relu")(x)
    z=Conv2D(filters=256,kernel_size=(1,1), padding="same", activation="relu")(z)
    z=DepthwiseConv2D(kernel_size=(2,2),strides=(2,2), padding="same", activation="relu")(z)
    
    #=============================================================================
    x=Conv2D(filters=576, kernel_size=(3,3), padding="same", activation="relu")(x)
    x=Conv2D(filters=576, kernel_size=(3,3), padding="same", activation="relu")(x)
    x=Conv2D(filters=576,kernel_size=(3,3), padding="same", activation="relu")(x)
    x=MaxPool2D(pool_size=(2,2),strides=(2,2))(x)
    
    x=layers.Concatenate()([x,y,z])
    x=Conv2D(filters=896, kernel_size=(3,3), activation="relu")(x)
    x=Conv2D(filters=896, kernel_size=(2,2), padding="same", activation="relu")(x)
    x=DepthwiseConv2D(kernel_size=(3,3), padding="same", activation="relu")(x)
    x=MaxPool2D(pool_size=(2,2),strides=(2,2))(x)
    return x    

inputs=keras.Input(shape=(512,512,3))
y=inflow(inputs)
y=midflow(y)
outputs=outflow(y)
Model= keras.Model(inputs,outputs)
Model.summary()

In [ ]:
Model.compile(
       optimizer=tf.keras.optimizers.Adam(0.000005),
       loss='categorical_crossentropy',
       metrics=['accuracy']
)
#Model.load_weights("transfer.h5")
Model.fit(train_data,epochs=3
          ,validation_data=val_data
         )
Model.save("transfer.h5")

In [ ]:
Model.compile(
       optimizer=tf.keras.optimizers.Adam(0.000004),
       loss='categorical_crossentropy',
       metrics=['accuracy']
)
#Model.load_weights("transfer.h5")
Model.fit(train_data,epochs=4
          ,validation_data=val_data
         )
Model.save("transfer.h5")

In [ ]:
Model.compile(
   optimizer=tf.keras.optimizers.Adam(0.0000015),
   loss='categorical_crossentropy',
   metrics=['accuracy']

)
Model.load_weights("transfer.h5")
Model.fit(train_data,epochs=1,validation_data=val_data)
Model.save("transfer.h5")

In [24]:

def classify(Id):
    x=predictwithId(Id)
    if x>0.995:
        return 0.99
    if x<0.0025:
        return 0.005
    return float(x)
    

def predictwithId(Id):
    path='../input/dl-hack-track-1-cv/test/'+str(Id)+".png"
    image = tf.keras.preprocessing.image.load_img(path,
                                              grayscale=False, 
                                              color_mode="rgb", 
                                              target_size=None,
                                              interpolation="nearest")
    input_arr = keras.preprocessing.image.img_to_array(image)
    input_arr = np.array(input_arr)/255  # Convert single image to a batch.
    result=model.predict(np.reshape(input_arr,(1,512,512,3)))[:,1]
    return result


In [25]:
path='../input/dl-hack-track-1-cv/train/fake/'+str(4)+".png"
image = tf.keras.preprocessing.image.load_img(path,
                                              grayscale=False, 
                                              color_mode="rgb", 
                                              target_size=None,
                                              interpolation="nearest")
input_arr = keras.preprocessing.image.img_to_array(image)
input_arr = np.array(input_arr)/255  # Convert single image to a batch.
result=model.predict(np.reshape(input_arr,(1,512,512,3)))[:,1]
float(result)

0.0009194078156724572

In [23]:
submission=pd.read_csv("../input/dl-hack-track-1-cv/sample_submission.csv")
submission['p_real']=submission['id'].apply(classify)
print(len(submission))
submission.set_index('id').to_csv('sample_submission.csv')

5000


<pre>
Model.compile(
       optimizer=tf.keras.optimizers.Adam(0.00005),
       loss='categorical_crossentropy',
       metrics=['accuracy']
)
#Model.load_weights("transfer.h5")
Model.fit(train_data,epochs=3
          ,validation_data=val_data
         )
Model.save("transfer.h5")

Epoch 1/3
219/219 [==============================] - 1196s 5s/step - loss: 0.5817 - accuracy: 0.6411 - val_loss: 0.6919 - val_accuracy: 0.5164
Epoch 2/3
219/219 [==============================] - 1223s 6s/step - loss: 0.2059 - accuracy: 0.9193 - val_loss: 0.2684 - val_accuracy: 0.8927
Epoch 3/3
219/219 [==============================] - 1177s 5s/step - loss: 0.1251 - accuracy: 0.9529 - val_loss: 0.1380 - val_accuracy: 0.9453
</pre>
